In [1]:
import json
from IPython.display import HTML, display
import re
import pandas as pd
import pickle
from pathlib import Path
import json
from datetime import date

### !pip install lxml

In [12]:
fname = 'examples/SiameseImage_working.ipynb'
nb_name = 'nb_save'
nb_path = Path(nb_name+'.ipynb')

In [3]:
main_dic = json.load(open(fname,'r',encoding="utf-8"))
print(main_dic.keys())

dict_keys(['cells', 'metadata', 'nbformat', 'nbformat_minor'])


In [4]:
cells = main_dic['cells']

In [5]:
def get_cell_from_exCount(cells, count):
    for cell in cells:
        if cell['cell_type'] == 'code' and cell['execution_count'] == count:
            return cell
    raise(ValueError('Count is higher than number of cells'))

In [7]:
def get_cell_from_identifier(cells, id):
    # First line of the source code/text starts with identifier
    for cell in cells:
        src = cell['source']
        #print(src)
        if src[0].startswith('#ID:'+id):
            return cell
        
    raise(ValueError(f'No cell cointaining the identifier: {id}'))

In [8]:
def get_output(cell):
    
    if 'outputs' in cell.keys():
        for output in cell['outputs']:
            if 'text/html' in output['data'].keys():
                return ' '.join([line.strip() for line in output['data']['text/html']])
            
            raise(ValueError('Cell does not contain text/html output'))
            
    raise(ValueError('Cell does not contain any output'))

In [9]:
#ID:experiment1
df_in = pd.read_csv('save/test.csv', header=None)
df_in.head()

,0,1,2,3,4
0,0,0.495539,0.534035,0.724315,00:10
1,1,0.497976,0.533547,0.719178,00:10
2,2,0.489686,0.534120,0.722603,00:10
3,3,0.497192,0.534630,0.726027,00:10
4,4,0.496531,0.531191,0.729452,00:10


In [14]:
cells = json.load(open(nb_path,'r',encoding='utf-8'))['cells']
cell = get_cell_from_identifier(cells, id="experiment1")

In [16]:
def add_to_collection(id, content, description, parameters, contet_type):
    
    filename = Path('./save/collection.pkl')
    
    if filename.exists():
        with open(filename, 'rb') as f:
            l = pickle.load(f)
    else:
        l = []
    
    d = dict(
        id = id,
        content = content,
        description = description,
        parameters = parameters,
        date = date.today().strftime("%d/%m/%Y")
    )
    
    #print(d)
    
    l += [d]
    
    #print(l)
    
    with open(filename, 'wb') as f:
        pickle.dump(l, f, )

In [18]:
content = get_output(cell)
add_to_collection(
    'experiment1',
    content,
    'result of experiment1',
    'arch: bla, model: bub, freeze: false,...',
    'table'
)

In [19]:
def show_collection():
    filename = Path('./save/collection.pkl')
    
    with open(filename, 'rb') as f:
        collection = pickle.load(f)
        
    for x in collection:
        print(f"{x['date']}: {x['description']}")
        print(x['parameters'])
        display(HTML(x['content']))

In [20]:
show_collection()

10/05/2021: this is where this comes from
{'arch': 'resnet34', 'epochs': 7, 'stage': 'frozen'}


epoch,train_loss,valid_loss,accuracy,time
0,0.495539,0.534035,0.724315,00:10
1,0.497976,0.533547,0.719178,00:10
2,0.489686,0.534120,0.722603,00:10
3,0.497192,0.534630,0.726027,00:10
4,0.496531,0.531191,0.729452,00:10


10/05/2021: this is where this comes from
{'arch': 'resnet34', 'epochs': 7, 'stage': 'frozen'}


epoch,train_loss,valid_loss,accuracy,time
0,0.495539,0.534035,0.724315,00:10
1,0.497976,0.533547,0.719178,00:10
2,0.489686,0.534120,0.722603,00:10
3,0.497192,0.534630,0.726027,00:10
4,0.496531,0.531191,0.729452,00:10


10/05/2021: this is where this comes from
{'arch': 'resnet34', 'epochs': 7, 'stage': 'frozen'}


epoch,train_loss,valid_loss,accuracy,time
0,0.495539,0.534035,0.724315,00:10
1,0.497976,0.533547,0.719178,00:10
2,0.489686,0.534120,0.722603,00:10
3,0.497192,0.534630,0.726027,00:10
4,0.496531,0.531191,0.729452,00:10


11/05/2021: result of experiment1
arch: bla, model: bub, freeze: false,...


,0,1,2,3,4
0,0,0.495539,0.534035,0.724315,00:10
1,1,0.497976,0.533547,0.719178,00:10
2,2,0.489686,0.534120,0.722603,00:10
3,3,0.497192,0.534630,0.726027,00:10
4,4,0.496531,0.531191,0.729452,00:10


In [21]:
def html_to_table(html):
    
    return pd.read_html(html)[0]

In [22]:
def is_table(output):
    return re.match(r'^<table.*</table>', output) is not None

In [23]:
def save_table(output, filename, to='csv'):
    from pathlib import Path
    import pandas as pd
    
    assert(is_table(output)), 'Content is no html table'
    
    path = Path('save')
    path.mkdir(exist_ok=True)
    
    if to == 'csv':
        filename += '.csv'
        pd.read_html(output)[0].to_csv(path / filename, index = False, header = False)